## Berkeley Segmentation Dataset 

<a href="https://www.kaggle.com/datasets/balraj98/berkeley-segmentation-dataset-500-bsds500">Buraya tıkladığınızda dataya ulaşabilirsiniz</a>

In [3]:
import cv2
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape

In [4]:
labels=os.listdir("images") 
img_path='images/'

In [5]:
#del labels[0]

In [6]:
labels

['test', 'train', 'val']

In [7]:
img_list=[]
label_list=[]
for label in labels:
    for img_file in os.listdir(img_path+label):
        img_list.append(img_path+label+"/"+img_file)
        label_list.append(label)

In [8]:
df=pd.DataFrame({'img':img_list,'label':label_list})

In [9]:
df.head()

,img,label
0,images/test/100007.jpg,test
1,images/test/100039.jpg,test
2,images/test/100099.jpg,test
3,images/test/10081.jpg,test
4,images/test/101027.jpg,test


In [10]:
df.sample()

,img,label
473,images/val/376043.jpg,val


In [11]:
d = {'test': 0, 'train': 1}
df['encode_label']=df['label'].map(d)

In [12]:
x=[]
for index, img_path in enumerate(df['img']):
    img = cv2.imread(img_path)

    if img is None:
        df.drop(index, inplace=True) 
        continue

    img = cv2.resize(img, (70, 70))
    img = img / 255.0 # normalie et 
    x.append(img) # listeye ekle 

In [13]:
x=np.array(x) # rakam ve diziye çevir

In [14]:
y=df[['encode_label']]

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y, random_state=42, test_size=0.20)

In [16]:
model=Sequential()
model.add(Input(shape=(70,70,3)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(3,activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [17]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,verbose=1,batch_size=32)

Epoch 1/50
13/13 [==============================] - 3s 27ms/step - loss: nan - accuracy: 0.3650 - val_loss: nan - val_accuracy: 0.4200
Epoch 2/50
13/13 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.3950 - val_loss: nan - val_accuracy: 0.4200
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.3950 - val_loss: nan - val_accuracy: 0.4200
Epoch 4/50
13/13 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.3950 - val_loss: nan - val_accuracy: 0.4200
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.3950 - val_loss: nan - val_accuracy: 0.4200
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.3950 - val_loss: nan - val_accuracy: 0.4200
Epoch 7/50
13/13 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.3950 - val_loss: nan - val_accuracy: 0.4200
Epoch 8/50
13/13 [==============================] - 0s 7ms/st

In [18]:
model.save('Berkeley_Segmentation_Dataset.h5')

In [19]:
# Transfer Learning

In [20]:
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
data_dir='images'
img_width,img_heigth=224,224

train_datagen=ImageDataGenerator(rescale=1/255, validation_split=0.20)

train_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_width,img_heigth),
                                class_mode='sparse', subset='training')

test_datagen=ImageDataGenerator(rescale=1/255)
test_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_width,img_heigth),
                                class_mode='sparse', subset='validation')

base_model=VGG16(weights='imagenet', input_shape=(img_width,img_heigth,3),include_top=False)

model=Sequential()
model.add(base_model)
for layer in base_model.layers:
    layer.trainable=False
    
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(train_datagenerator,epochs=10,validation_data=test_datagenerator)

Found 400 images belonging to 3 classes.
Found 100 images belonging to 3 classes.
Epoch 1/10
13/13 [==============================] - 11s 491ms/step - loss: 10.1013 - accuracy: 0.3775 - val_loss: 2.2267 - val_accuracy: 0.4400
Epoch 2/10
13/13 [==============================] - 2s 156ms/step - loss: 1.9295 - accuracy: 0.5200 - val_loss: 2.0390 - val_accuracy: 0.2800
Epoch 3/10
13/13 [==============================] - 2s 153ms/step - loss: 0.8398 - accuracy: 0.6825 - val_loss: 1.8490 - val_accuracy: 0.3700
Epoch 4/10
13/13 [==============================] - 2s 155ms/step - loss: 0.3711 - accuracy: 0.8525 - val_loss: 1.5970 - val_accuracy: 0.4200
Epoch 5/10
13/13 [==============================] - 2s 154ms/step - loss: 0.1784 - accuracy: 0.9475 - val_loss: 1.6121 - val_accuracy: 0.4000
Epoch 6/10
13/13 [==============================] - 2s 154ms/step - loss: 0.1240 - accuracy: 0.9750 - val_loss: 1.8348 - val_accuracy: 0.3000
Epoch 7/10
13/13 [==============================] - 2s 155ms/ste

In [22]:
model.save('Berkeley_VGG16.h5')